# Hydrogen depassivation lithography

In [2]:
Nx=1 
Ny_max=2

U11=1 # nearest neighbor potential in first layer
U22=1 # nearest neighbor potential term in 2nd layer
Usi=1# nearest neighbor potential betweeen hydrogen and silicon atom

t22=1 # nearest nighbor hopping in 2nd layer
t11=1 # nearest neighbor hopping in first layer
t_si= # nearest neighor hopping between silicon and hydrogen atom
t_b1=1 # loss/gain at boundary 1
t_b2=2 # loss/gain at boundary 2
t_b3=3 # loss/gain at boundary 3
t_b4=4; # loss/gain at boundary 4

In [6]:
it=Iterators.product(ntuple(_ -> 0:1, 1+Nx*Ny_max*6)...);
p=collect(it); # possible states with non conserving particle number

In [10]:
# Generating a matrix with zeros.
H=zeros(2^(1+Nx*Ny_max*6),2^(1+Nx*Ny_max*6)); # 6 nearest neighbours.
size(H)

(8192, 8192)

In [14]:
bonds = [
        # first layer.
        [1,2],[2,3],[3,4],[7,8],[8,9],[9,10],
        # second layer.
        [5,6],[11,12],
        # first to second layer.
        [2,5],[4,6],[8,11],[10,12],
        # second to third layer.
        [8,5],[10,6]
        ];

In [23]:
layer_1 = [];
for Ny=1:Ny_max
    for k=6*(Nx)*(Ny-1)+1:6*(Nx)*(Ny-1)+4*Nx-1
        push!(layer_1,[k,k+1])
    end
end

In [32]:
layer_2 = [];
for Ny=1:Ny_max
    for k=6*Nx*(Ny-1)+4*Nx+1:2:6*Nx*(Ny-1)+4*Nx+2*Nx-1
        push!(layer_2,[k,k+1])
    end
end
layer_2

2-element Vector{Any}:
 [5, 6]
 [11, 12]

In [25]:
layer_1_2 = [];
for Ny=1:Ny_max
    for n=2:2:4*Nx
        push!(layer_1_2,[Nx*6*(Ny-1)+n,Nx*6*(Ny-1)+4*Nx+Int(n/2)])
        #println(Nx*6*(Ny-1)+n)
        #println(Nx*6*(Ny-1)+4*Nx+Int(n/2))
    end
end
layer_1_2

4-element Vector{Any}:
 [2, 5]
 [4, 6]
 [8, 11]
 [10, 12]

In [30]:
layer_2_3 = [];
for Ny=2:Ny_max
    for n1=2:2:4*Nx
        push!(layer_2_3,[Nx*6*(Ny-1)+n1,Nx*6*(Ny-2)+4*Nx+Int(n1/2)])
        #println(Nx*6*(Ny-1)+n1)
        #println(Nx*6*(Ny-2)+4*Nx+Int(n1/2))
    end
end
layer_2_3

2-element Vector{Any}:
 [8, 5]
 [10, 6]

## Imagine I'm given a list bonds = [[1,2],[2,3],etc]

In [11]:
# Imagine I'm given a list bonds = [[1,2],[2,3],etc]

for n=1:2^(1+Nx*Ny_max*6) # Loop over basis states

    for j=1:length(bonds)
        
        site1=bonds[j][1]
        site2=bonds[j][2]
        
        #=
        Assuming NN interaction between any two layers are same (U11).
        =#
        
        #= NN interaction term has no Hermitian conjugate=#
        H[n,n]+=U11*p[n][site1]*p[n][site2] # NN Interaction term
        
        
        #= NN hopping has Hermitian conjugate =#
        if p[n][site1] == 1 && p[n][site2]==0 # NN hopping
            q=collect(p[n])
            q[site1]=0
            q[site2]=1
            
            phase1=(-1)^(sum(p[n][site1:6*Nx*Ny_max+1]))
            phase2=1
            
            if site2 < 6*Nx*Ny_max+1 
                phase2=(-1)^(sum(p[n][site2:6*Nx*Ny_max+1]))
            end
            
            for n1=1:2^(6*Nx*Ny_max+1)
                if collect(p[n1])==q
                    H[n,n1]+=t22*phase1*phase2
                    H[n1,n]+=t22*phase1*phase2 # hermitian conjugate
                end
            end
        end
    end
end